# Download 'horizontal' + 'vertical' Landsat ARD tiles via Earth Explorer API

In [1]:
import requests
import json
import getpass
import os
import tarfile
from urllib.parse import urlparse
import shutil

In [2]:
def get_apikey(username, password):
    payload = dict(username=username, password=password, catalogId='EE', authType='EROS')
    data = dict(jsonRequest=json.dumps(payload))
    r = requests.post('https://earthexplorer.usgs.gov/inventory/json/v/1.4.0/login', data=data)
    creds = r.json()
    apiKey = creds['data']
    return apiKey

In [3]:
username = 'scottyhq'
password = getpass.getpass()

 ··········


In [4]:
apiKey = get_apikey(username, password)

In [5]:
def get_scenes(region, horizontal, vertical):
    myfilter = {
            "filterType": "and", 
            "childFilters": [
                {"filterType":"value","fieldId":21789,"value":region}, #Grid Region
                {"filterType":"value","fieldId":21787,"value":horizontal}, #Horizontal
                {"filterType":"value","fieldId":21788,"value":vertical}, #Vertical
            ]
        }

    request_code = 'search'
    baseurl = f'https://earthexplorer.usgs.gov/inventory/json/v/1.4.0/{request_code}'
    payload = {'apiKey':apiKey, 
               'datasetName': 'ARD_TILE', 
               'additionalCriteria': myfilter,  
               'maxResults':10000}
    data = dict(jsonRequest=json.dumps(payload))
    r = requests.get(baseurl, params=data)
    response = r.json()
    return response

In [6]:
# Query USGS Earth Explorer API for scenes
region='AK'
horizontal='008'
vertical='003'
response = get_scenes(region, horizontal, vertical)

In [7]:
print('Found {} scenes'.format(response['data']['totalHits']))
print('Returned {} scenes'.format(response['data']['numberReturned']))
print('First acquisition: ', response['data']['results'][0]['acquisitionDate'])
print('Last acquisition: ', response['data']['results'][-1]['acquisitionDate'])

Found 2595 scenes
Returned 2595 scenes
First acquisition:  1984-06-18
Last acquisition:  2019-03-01


In [8]:
def get_url(entityId, apiKey, product='ST'):
    request_code = 'download'
    baseurl = f'https://earthexplorer.usgs.gov/inventory/json/v/1.4.0/{request_code}'
    payload = {'apiKey':apiKey, 
               'datasetName': 'ARD_TILE', 
               'entityIds':[entityId],
               'products':[product]} #these are 'download codes' from above
    data = dict(jsonRequest=json.dumps(payload))
    r = requests.get(baseurl, params=data)
    download_url = r.json()['data'][0]['url']
    p = urlparse(download_url)
    name = os.path.basename(p.path)
    return name, download_url

In [9]:
def download(filename, url):
    chunks = 1024*1024
    with requests.get(url, stream=True) as r:
        with open(filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=chunks):
                f.write(chunk)

In [10]:
# grab a file for testing
ids = [x['entityId'] for x in response['data']['results']]

In [11]:
# Download all in serial (try first 5)
success = []
fail = []
for entityId in ids[:5]:
    print(entityId)
    try:
        tarname, url = get_url(entityId, apiKey)
        download(tarname, url)
        success.append(entityId)
    except IndexError:
        print('no downloadable product for that id...')
        fail.append(entityId)

LT05_AK_008003_19840618_C01_V01
LT05_AK_008003_19840711_C01_V01
no downloadable product for that id...
LT05_AK_008003_19840720_C01_V01
no downloadable product for that id...
LT05_AK_008003_19840729_C01_V01
no downloadable product for that id...
LT05_AK_008003_19840812_C01_V01
